In [2]:
import os

from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser,JsonOutputParser

load_dotenv()  # 自动加载 .env 文件

print(os.getenv("LLM_MODEL"))

glm-4.6


# 1. 获取大模型

In [6]:
# print(os.getenv("OPENAI_API_KEY"))
# print(os.getenv("OPENAI_BASE_URL"))

llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL"),
    model=os.getenv("LLM_MODEL"),
    temperature=float(os.getenv("LLM_TEMPERATURE"))
)

response = llm.invoke("RAG 技术的核心流程")
print(response.content)


RAG（Retrieval-Augmented Generation，检索增强生成）是一种结合**检索外部知识库**与**生成式AI**的技术框架，其核心目标是解决大语言模型（LLM）固有知识的局限性（如过时、不完整），通过动态获取实时/领域特定信息来提升回答的准确性与可靠性。以下是RAG技术的核心流程拆解：


### **一、知识库构建与预处理**
RAG的第一步是将**外部知识源**转化为可检索的结构化数据，关键步骤包括：
1. **数据采集**：  
   收集待注入的外部知识，来源可以是文档（PDF、Word）、网页、数据库、API接口等。
2. **数据清洗与分割（Chunking）**：  
   对原始数据进行去噪（如去除格式标签、广告）、分块处理（将长文本切分为固定大小的“片段”）。  
   - 分块目的：避免超出LLM的上下文窗口限制，同时保留语义完整性（如按段落、章节或语义边界分割）。
3. **向量化（Embedding）**：  
   使用**嵌入模型**（如OpenAI的`text-embedding-ada-002`、Sentence-BERT、BERT等）将每个文本块转换为高维向量（ embedding ）。  
   - 原理：将文本映射到向量空间，使语义相近的文本块在向量空间中距离较近，便于后续相似度检索。


### **二、向量数据库索引构建**
将向量化后的文本块存储到**向量数据库**（Vector Database）中，并建立高效索引结构，以支持快速检索。常见工具包括：
- 开源向量数据库：FAISS（Facebook AI Similarity Search）、ChromaDB、Milvus；
- 商业服务：Pinecone、Weaviate。  

索引的作用是通过空间划分（如HNSW、IVF）加速向量相似度搜索，降低检索延迟。


### **三、查询处理与检索**
当用户提交问题时，系统执行以下步骤：
1. **查询向量化**：  
   使用与知识库一致的**嵌入模型**，将用户问题转换为向量（query embedding）。
2. **相似度检索**：  
   在向量数据库中搜索与查询向量**语义最相似**的Top-K个文本块（K为预设的检索数量，如5-20条）。  
   - 核心算法：余弦相似度

# 2.使用提示词模板

In [7]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是世界级的技术文档编写者"),
    ("user", "{input}") # {input}为变量
])

llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL"),
    model=os.getenv("LLM_MODEL"),
    temperature=float(os.getenv("LLM_TEMPERATURE"))
)

chain = prompt | llm # 把prompt和具体lm的调用和在一起
message = chain.invoke({"input": "大模型中的LangChain是什么?"})
print(message)

content='\n好的，请坐好。作为一名世界级的技术文档编写者，我将为您清晰、全面、且深入浅出地解析 LangChain。\n\n---\n\n### **LangChain：大模型应用开发的“瑞士军刀”**\n\n#### **1. 核心定义：LangChain 是什么？**\n\n想象一下，你拥有一个极其聪明但没有四肢、没有记忆、也无法与外界直接交流的大脑（这就是大语言模型，LLM）。它知道很多事，但你问它今天的天气，它不知道；你让它帮你查一封上周的邮件，它也做不到。\n\n**LangChain 就是为这个“大脑”量身打造的“身体”和“工具箱”。**\n\n它是一个**开源框架**，专门用于简化基于大语言模型（LLM）的应用程序开发。它不创造模型，而是**连接**和**赋能**模型，让开发者能够像搭乐高积木一样，将LLM的各种能力与外部数据源、API和计算能力组合起来，构建出功能强大的复杂应用。\n\n**一句话总结：LangChain 是一个开发框架，旨在通过“链”式调用各种组件，将LLM的能力扩展到更广阔的应用场景。**\n\n---\n\n#### **2. 为什么需要 LangChain？解决了什么痛点？**\n\n原生的大语言模型存在以下核心限制，而 LangChain 恰好逐一击破：\n\n| LLM 的原生限制 | LangChain 的解决方案 |\n| :--- | :--- |\n| **缺乏上下文记忆**：每次对话都是独立的，无法记住历史信息。 | **记忆**：提供多种记忆组件，让应用能够存储和回顾对话历史，实现连贯的交流。 |\n| **知识陈旧**：训练数据有截止日期，不知道之后发生的新事。 | **索引与检索**：连接外部数据库（如向量数据库），让LLM能够访问和利用最新的私有或公开数据。 |\n| **无法执行操作**：只能生成文本，不能调用API、运行代码或浏览网页。 | **工具与代理**：赋予LLM使用“工具”（如搜索引擎、计算器、Python解释器）的能力，使其能自主规划并执行任务。 |\n| **开发流程繁琐**：每次与LLM交互都需要手动处理提示词、解析输出、连接API等。 | **链**：将多个步骤（如：提示模板 -> LLM -> 输出解析器）封装成一个可复用的“链”，极大简化开发流程。 |\n| **输出不

In [4]:
print(type(message)) # 输出message类型

<class 'langchain_core.messages.ai.AIMessage'>


# 3.使用输出解析器

In [9]:
# 在2的基础上
# 使用输出解析器
output_parser = JsonOutputParser()
chain = prompt | llm | output_parser # 将其添加到上一个链中

# chain.invoke({"input": "LangChain是什么？使用JSON格式回复，问题用question，回答用answer"}) # 使用方式1
chain.invoke({"input": "LangChain是什么？使用JSON格式回复", "format_instructions": output_parser.get_format_instructions()}) # 使用方式2


{'langChainDefinition': {'overview': {'summary': 'LangChain 是一个开源框架，旨在简化基于大型语言模型的应用程序开发。',
   'detailedExplanation': '它提供了一套工具、组件和接口，让开发者能够将 LLM 与外部数据源、API 和计算能力无缝结合，从而构建出功能强大、复杂的应用程序，而无需从零开始编写大量胶水代码。'},
  'coreConcepts': [{'name': '模型',
    'englishTerm': 'Models',
    'description': '作为与各种大型语言模型（如 OpenAI、Hugging Face Hub 等）交互的接口，支持多种模型类型，包括 LLMs、聊天模型 和文本嵌入模型。'},
   {'name': '提示词',
    'englishTerm': 'Prompts',
    'description': '管理与优化输入给 LLM 的指令。提示词模板 允许开发者动态生成提示词，实现输入的标准化和复用。'},
   {'name': '链',
    'englishTerm': 'Chains',
    'description': 'LangChain 的核心概念之一。它将多个组件（如模型、提示词）按特定顺序连接起来，形成一个处理流程，以完成更复杂的任务。'},
   {'name': '索引',
    'englishTerm': 'Indexes',
    'description': '用于构建和查询外部知识库的结构化方法。它涉及文档加载、文本分割、向量化和存储，是实现检索增强生成（RAG）的基础。'},
   {'name': '代理',
    'englishTerm': 'Agents',
    'description': '最具动态性的组件。代理使用 LLM 作为“推理引擎”，根据任务目标自主决定调用哪些工具（如搜索引擎、计算器、API），并观察结果，循环执行直到任务完成。'},
   {'name': '记忆',
    'englishTerm': 'Memory',
    'description': '为链和代理提供持久化状态的能力，使其能够在多次交互之间记住信息，是构建有上

# 4. 使用向量存储

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
import bs4

loader = WebBaseLoader(
    web_path="https://www.gov.cn/yaowen/liebiao/202511/content_7049693.htm",
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(id="UCAP-CONTENT"))
)
docs = loader.load()
# print(docs)

# 使用api调用嵌入模型
embeddings = OpenAIEmbeddings(model='Embedding-3')
# 使用分割器分割文档
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(docs)
print(documents)
print(len(documents))
# 向量存储 embeddings 会将 documents 中的每个文本片段转换为向量，并将这些向量存储在 FAISS 向量数据库中
vector = FAISS.from_documents(documents, embeddings)

[Document(metadata={'source': 'https://www.gov.cn/yaowen/liebiao/202511/content_7049693.htm'}, page_content='李强主持召开国务院常务会议听取推动高质量发展综合督查情况汇报部署推进基本医疗保险省级统筹工作审议通过《全民阅读促进条例（草案）》讨论《中华人民共和国注册会计师法（修正草案）》新华社北京11月27日电'), Document(metadata={'source': 'https://www.gov.cn/yaowen/liebiao/202511/content_7049693.htm'}, page_content='国务院总理李强11月27日主持召开国务院常务会议，听取推动高质量发展综合督查情况汇报，部署推进基本医疗保险省级统筹工作，审议通过《全民阅读促进条例（草案）》，讨论《中华人民共和国注册会计师法（修正草案）》。会议指出，这次推动高质量发展综合督查，围绕深入贯彻落实习近平总书记重要指示批示精神和党中央决策部署，促进各地区各部门更加扎实做好经济社会发展工作，取得积极成效。要切实用好综合督查成果，对督查发现的主要问题，督促有关部门和地方逐项抓好整改落实。对提出的督办意见和政策建议，要深入对照分析，及时完善政策、改进工作。要聚焦重点领域薄弱环节，举一反三开展自查自纠，由解决一个问题向解决一类问题延伸，巩固完善抓整改促落实长效机制。会议指出，推进基本医保省级统筹是健全全民医保制度的重要举措，有利于更好发挥保险互助共济优势，增强医保制度保障能力。要指导各地因地制宜分类施策，合理确定省级统筹的基金管理模式，提高基金使用效率。要协调推进基本医保省级统筹和分级诊疗制度建设，加快推动优质医疗资源均衡布局，加强基层医疗服务能力建设，完善医保支付和服务价格调节机制，更好满足群众就医需求。会议指出，要从法律上为全'), Document(metadata={'source': 'https://www.gov.cn/yaowen/liebiao/202511/content_7049693.htm'}, page_content='基层医疗服务能力建设，完善医保支付和服务价格调节机制，更好满足群众就医需求。会议指出，要从法律上为全民阅读提供保障，让人民群众通过阅读不断

# 5.RAG(检索增强生成)

In [8]:
from langchain_core.prompts import PromptTemplate

retriever = vector.as_retriever()
retriever.search_kwargs = {"k": 3}
docs = retriever.invoke("会议推出了什么政策？")

for i,doc in enumerate(docs):
    print(f'※第{i+1}条策略')
    print(doc)

# 定义提示词模板
prompt_template = """
你是一个问答机器人，
你的任务是根据下述给定的已知信息回答用户问题。
确保你的回复完全依据下述已知信息。不要编造答案。
如果下述已知信息不足以回答用户的问题，请直接回复”我无法回答您的问题“。

已知信息：
{info}

用户问：
{question}
"""

# 得到提示词模板对象
template = PromptTemplate.from_template(prompt_template)

# 得到提示词对象
prompt = template.format(info=docs, question='会议推出了什么政策？')

# 调用LLM
response = llm.invoke(prompt)
print(response.content)

※第1条策略
page_content='国务院总理李强11月27日主持召开国务院常务会议，听取推动高质量发展综合督查情况汇报，部署推进基本医疗保险省级统筹工作，审议通过《全民阅读促进条例（草案）》，讨论《中华人民共和国注册会计师法（修正草案）》。会议指出，这次推动高质量发展综合督查，围绕深入贯彻落实习近平总书记重要指示批示精神和党中央决策部署，促进各地区各部门更加扎实做好经济社会发展工作，取得积极成效。要切实用好综合督查成果，对督查发现的主要问题，督促有关部门和地方逐项抓好整改落实。对提出的督办意见和政策建议，要深入对照分析，及时完善政策、改进工作。要聚焦重点领域薄弱环节，举一反三开展自查自纠，由解决一个问题向解决一类问题延伸，巩固完善抓整改促落实长效机制。会议指出，推进基本医保省级统筹是健全全民医保制度的重要举措，有利于更好发挥保险互助共济优势，增强医保制度保障能力。要指导各地因地制宜分类施策，合理确定省级统筹的基金管理模式，提高基金使用效率。要协调推进基本医保省级统筹和分级诊疗制度建设，加快推动优质医疗资源均衡布局，加强基层医疗服务能力建设，完善医保支付和服务价格调节机制，更好满足群众就医需求。会议指出，要从法律上为全' metadata={'source': 'https://www.gov.cn/yaowen/liebiao/202511/content_7049693.htm'}
※第2条策略
page_content='基层医疗服务能力建设，完善医保支付和服务价格调节机制，更好满足群众就医需求。会议指出，要从法律上为全民阅读提供保障，让人民群众通过阅读不断获取知识、增长智慧。要加大优质内容供给，建设用好全民阅读设施，完善青少年、老年人等不同人群阅读服务，鼓励促进全民阅读的新技术、新载体、新设施等开发应用，创新开展丰富多样的阅读推广活动，增强全民族思想道德素质和科学文化素养，提高全社会文明程度，为推进中国式现代化注入强大精神力量。会议讨论并原则通过《中华人民共和国注册会计师法（修正草案）》，决定将草案提请全国人大常委会审议。会议指出，要强化法律约束，进一步加强行业监管和财会监督，严肃查处违法违规行为，促进注册会计师诚实守信、依法履责，推动注册会计师行业健康发展，在维护市场经济秩序、保护投资者权益、提升企业经营水平等方面发挥应有作用。会议还研究了其他事项。

# 6.使用Agent

In [10]:
from langchain.tools.retriever import create_retriever_tool
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# 检索器工具
retriever_tool = create_retriever_tool(
    retriever,
    'CivilCodeRetriever',
    '搜索有关中华人民共和国民法典的信息。关于中华人民共和国民法典的任何问题，您必须使用此工具！'
)

tools = [retriever_tool]

prompt = hub.pull('hwchase17/openai-functions-agent')

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 运行代理
agent_executor.invoke({"input": "建设用地使用权是什么?"})



> Entering new AgentExecutor chain...

好的，我们来详细、通俗地解释一下“建设用地使用权”这个概念。

简单来说，**建设用地使用权**，就是**个人或单位在国有的土地上建造建筑物、构筑物以及其他附属设施的权利**。

这是一个在中国特色土地制度下非常核心的法律概念。为了更好地理解它，我们可以从以下几个方面来拆解：

---

### 一、核心定义：它不是“所有权”，而是“使用权”

这是最关键的一点。

*   **土地所有权**：根据中国宪法，城市的土地属于**国家所有**；农村和城市郊区的土地，除法律规定属于国家所有的以外，属于**农民集体所有**。这意味着，在中国，私人或企业**不能拥有土地的所有权**。
*   **建设用地使用权**：这是一种**用益物权**。通俗地讲，就是国家作为“地主”，把土地“租”给你使用一段时间（这个“租”是法律上的、有偿的、长期的），让你在这块地上盖房子、办工厂等。你盖出来的房子、厂房等**建筑物的所有权是你的**，但你脚下的土地，你拥有的只是“使用权”。

**一个简单的比喻：**
就像你租了一个长租公寓70年。你有权在这个公寓里生活、装修，甚至可以把“租约”转给别人，但公寓的产权始终属于房东。在这里，国家就是房东，土地是公寓，你获得的“建设用地使用权”就是那份受法律保护的、超长期的“租约”。

---

### 二、主要特征

1.  **客体是国有土地**：这种权利通常设立在**国家所有的土地**上。如果要在农村集体土地上搞建设，需要先由国家将土地征收为国有，然后再出让建设用地使用权。
2.  **目的明确**：只能用于**“建设”**，即建造建筑物、构筑物（如工厂、住宅、办公楼、商场、学校等）。不能用于耕种、放牧等。
3.  **有期限限制**：建设用地使用权是有年限的，不是永久的。根据用途不同，最高年限如下：
    *   **居住用地**：70年（这是大家最关心的）
    *   **工业用地**：50年
    *   **教育、科技、文化、卫生、体育用地**：50年
    *   **商业、旅游、娱乐用地**：40年
    *   **综合或其他用地**：50年
4.  **是一种财产权**：虽然只是使用权，但它具有巨大的经济价值，可以像其他财产一样进行**转让、

{'input': '建设用地使用权是什么?',
 'output': '\n好的，我们来详细、通俗地解释一下“建设用地使用权”这个概念。\n\n简单来说，**建设用地使用权**，就是**个人或单位在国有的土地上建造建筑物、构筑物以及其他附属设施的权利**。\n\n这是一个在中国特色土地制度下非常核心的法律概念。为了更好地理解它，我们可以从以下几个方面来拆解：\n\n---\n\n### 一、核心定义：它不是“所有权”，而是“使用权”\n\n这是最关键的一点。\n\n*   **土地所有权**：根据中国宪法，城市的土地属于**国家所有**；农村和城市郊区的土地，除法律规定属于国家所有的以外，属于**农民集体所有**。这意味着，在中国，私人或企业**不能拥有土地的所有权**。\n*   **建设用地使用权**：这是一种**用益物权**。通俗地讲，就是国家作为“地主”，把土地“租”给你使用一段时间（这个“租”是法律上的、有偿的、长期的），让你在这块地上盖房子、办工厂等。你盖出来的房子、厂房等**建筑物的所有权是你的**，但你脚下的土地，你拥有的只是“使用权”。\n\n**一个简单的比喻：**\n就像你租了一个长租公寓70年。你有权在这个公寓里生活、装修，甚至可以把“租约”转给别人，但公寓的产权始终属于房东。在这里，国家就是房东，土地是公寓，你获得的“建设用地使用权”就是那份受法律保护的、超长期的“租约”。\n\n---\n\n### 二、主要特征\n\n1.  **客体是国有土地**：这种权利通常设立在**国家所有的土地**上。如果要在农村集体土地上搞建设，需要先由国家将土地征收为国有，然后再出让建设用地使用权。\n2.  **目的明确**：只能用于**“建设”**，即建造建筑物、构筑物（如工厂、住宅、办公楼、商场、学校等）。不能用于耕种、放牧等。\n3.  **有期限限制**：建设用地使用权是有年限的，不是永久的。根据用途不同，最高年限如下：\n    *   **居住用地**：70年（这是大家最关心的）\n    *   **工业用地**：50年\n    *   **教育、科技、文化、卫生、体育用地**：50年\n    *   **商业、旅游、娱乐用地**：40年\n    *   **综合或其他用地**：50年\n4.  **是一种财产权**：虽然只是使用权，